In [3]:
import numpy as np

def Newton_system(F, DF, x0, N, eps):
    """
    Solve F(x) = 0 using Newton's method.
    
    Inputs:
      F: Function returning the residual vector F(x)
      DF: Function returning the Jacobian matrix DF(x)
      x0: Initial guess (vector)
      N: Maximum number of iterations
      eps: Convergence tolerance (stop when ||F(x)||_2 < eps)
    
    Output:
      x: Approximate solution vector
      steps: Number of steps taken (or -1 if convergence was not achieved)
    """
    x = x0.copy()            # Start with a copy of the initial guess
    F_val = F(x)             # Evaluate F at the current guess
    F_norm = np.linalg.norm(F_val, ord=2)  # Compute the Euclidean norm of F(x)
    steps = 0
    while F_norm > eps and steps < N:
        # Solve the linear system: DF(x)*s = F(x)
        s = np.linalg.solve(DF(x), F_val)
        x = x - s            # Update the solution: x_new = x - s
        F_val = F(x)         # Recompute the residual vector at the new guess
        F_norm = np.linalg.norm(F_val, ord=2)  # Update the norm of F(x)
        steps += 1
    if F_norm > eps:
        steps = -1           # Indicate failure to converge if tolerance not met
    return x, steps

def F_GPS_params(x, A, B, C, t, c):
    """
    Computes the residuals for the GPS system.
    
    For each satellite i:
      F_i(x,y,z,d) = sqrt((x - A_i)^2 + (y - B_i)^2 + (z - C_i)^2) - c*(t_i - d)
    
    x: Vector [x, y, z, d]
    A, B, C: Arrays of satellite positions (km)
    t: Array of travel times (seconds)
    c: Speed of light (km/s)
    
    Returns a 4-element vector of residuals.
    """
    distances = np.sqrt((x[0] - A)**2 + (x[1] - B)**2 + (x[2] - C)**2)
    return distances - c * (t - x[3])

def DF_GPS_params(x, A, B, C, c):
    """
    Computes the 4x4 Jacobian matrix for the GPS system.
    
    For each satellite i:
      ∂F_i/∂x = (x - A_i)/r_i,
      ∂F_i/∂y = (y - B_i)/r_i,
      ∂F_i/∂z = (z - C_i)/r_i,
      ∂F_i/∂d = c,
    where r_i = sqrt((x - A_i)^2 + (y - B_i)^2 + (z - C_i)^2).
    """
    J = np.zeros((4, 4))
    for i in range(4):
        r_i = np.sqrt((x[0] - A[i])**2 + (x[1] - B[i])**2 + (x[2] - C[i])**2)
        J[i, 0] = (x[0] - A[i]) / r_i
        J[i, 1] = (x[1] - B[i]) / r_i
        J[i, 2] = (x[2] - C[i]) / r_i
        J[i, 3] = c
    return J

# Given satellite positions (km) as provided in the assignment:
A = np.array([15600, 18760, 17610, 19170])
B = np.array([7540, 2750, 14630, 610])
C = np.array([20140, 18610, 13480, 18390])
# Given measured travel times (seconds):
t = np.array([0.07074, 0.07220, 0.07690, 0.07242])
c = 299792.458  # km/s

# Baseline receiver parameters: (x, y, z, d)
# Receiver at (0, 0, 6370) km, clock bias d = 0 s (per assignment, initial guess is d=0)
x0 = np.array([0.0, 0.0, 6370.0, 0.0])

solution, steps = Newton_system(
    lambda x: F_GPS_params(x, A, B, C, t, c),
    lambda x: DF_GPS_params(x, A, B, C, c),
    x0, 100, 1e-4)

print("Solution (Task 1):", solution)
print(f"x ≈ {solution[0]:.2f} km, y ≈ {solution[1]:.2f} km, z ≈ {solution[2]:.2f} km, d ≈ {solution[3]:.5f} s")
print("Number of steps:", steps)
print("Residual:", F_GPS_params(solution, A, B, C, t, c))


Solution (Task 1): [-4.17727094e+01 -1.67891940e+01  6.37005956e+03 -3.20156583e-03]
x ≈ -41.77 km, y ≈ -16.79 km, z ≈ 6370.06 km, d ≈ -0.00320 s
Number of steps: 2
Residual: [5.22850314e-08 4.41614247e-08 2.23371899e-08 4.76356945e-08]
